Create the table (or temp view) to store the parsed data

### Parsed Docs View
ai_parse_document: returns a variant object with all the elements in the pdf

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW parsed_docs as
Select
    row_number() over (order by path) as candidate_id,
    path as file_path,
    ai_parse_document(content) as parsed

From read_files(
  '/Volumes/workspace/default/resumes/*.pdf',
  format => 'BINARYFILE'
)

In [0]:
%sql
Select * from parsed_docs

### Extracted text view 
turns the parsed field into one big string. So we can plug it into llm in next step.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW extracted_text as 
Select 
  candidate_id, 
  file_path,
  concat_ws(
    '\n\n',
    transform(
      try_cast(parsed:document:elements as array<variant>),
      element -> try_cast(element:content as string)
    )
  ) as resume_text
  From parsed_docs
  where try_cast(parsed:error_status as string) is null

In [0]:
%sql
Select * from extracted_text

### raw response view
use llm to extract desired elements from the resume text. returns a string in json format

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW raw_response as 
Select candidate_id, 
file_path, 
ai_query(
  'databricks-meta-llama-3-3-70b-instruct',
  concat(
    'extract the following information from this resume and return only valid JSON with these exact key: ',
    '{"name": string, "email": string, "phone": string, "years_of_experience": number, ',
    '"skilles": [array of strings], "education": string, "previous_employers": [array of strings], ',
    '"current_job_title": string}. Do not include any message or summary before the json object.  Resume text: ',
    resume_text
  )
) as raw_json
From extracted_text
where resume_text is not null

In [0]:
%sql
Select * from raw_response

### Parse JSON
turn the raw_json into variant object

In [0]:
%sql
create or replace temporary view parsed_response as
Select 
candidate_id, 
file_path,
parse_json(raw_json) as parsed_data
from raw_response

### Insert into table
separated parsed_data into separate columns

In [0]:
%sql
create or replace temporary view final as
Select 
  parsed_data:name::STRING as name,
  parsed_data:email::STRING as email,
  parsed_data:phone::STRING as phone,
  parsed_data:years_of_experience::INT as years_of_experience,
  parsed_data:current_job_title::STRING as current_job_title,
  parsed_data:education::STRING as education,
  parsed_data:skills as skills,
  parsed_data:previous_employers as previous_employers
  From parsed_response

In [0]:
%sql
Select * from final

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.default.resume_data as (
  Select * from final
) 

In [0]:
%sql
Select * from workspace.default.resume_data